<h1> Tutorial 2.2 - The *lac* System with Spatial Resolution </h1>

This is like tutorial 2.1, except we investigate the system with spatial resolution. <br/>

The model presented here can be found in the article: [Noise Contributions in an Inducible Genetic Switch: A Whole-Cell Simulation Study](http://journals.plos.org/ploscompbiol/article?id=10.1371/journal.pcbi.1002010).

In [ ]:
import os
from pyLM import *
from pyLM.units import *
from pySTDLM import *
from pySTDLM.PostProcessing import *
from pySTDLM.StandardReactionSystems import *
from pySTDLM.StandardCells import *
%matplotlib inline

<h2> Defining a Spatial Simulation </h2>

Lattice Microbes spatial simulations sample the reaction diffusion master equation (RDME) by discretizing a phyiscal space into a rectangular grid of "lattice sites" and then stochastically simulating diffusion events between lattice sites and reactions within lattice sites.<br/>

Each lattice site can be defined as a different type of "region".  For instance, lattice sites may be membrane, or cytoplasm, or nucleus or extracellular space.  Each lattice site is described by only one type. Reactions are defined to take place in different regions and particles can diffuse among similar and among different lattice sites with different diffusion constants. particles can also be excluded from diffusing into or between certain site types. <br/>

When defining a spatial simulation, we need to define the extents of the simulation box and the lattice spacing. Here we set up a box that will hold a 1.8 $\mu$m long *E. coli* cell.  The box is divided into lattice sites that are 32 nm per side for a total lattice size of 32$\times$32$\times$64. <br/>

Documentaiton for the RDMESimulation can be found [here](http://www.scs.illinois.edu/schulten/lm/documentation/namespacepy_l_m_1_1_r_d_m_e.html).

In [ ]:
# Create our simulation object
latticeSpacing=nm(32)
sim=RDME.RDMESimulation(dimensions=micron(1.024,1.024,2.048), spacing=latticeSpacing, name="Lac Genetic Switch")

We next build a standard representation of an *E. coli* cell using the build in function [```buildCapsidCell()```](http://www.scs.illinois.edu/schulten/lm/documentation/classpy_l_m_1_1_r_d_m_e_1_1_r_d_m_e_simulation.html#a07a10a44bacbb0f781115425c0124d59). This builds a cell that is pill shaped; it is a cylinder with spheres on the ends. The cell is constructed to be 1.8 $\mu$m long (from tip to tip) and 500 nm wide. We define a membrane thickness that is roughly 2 lattice sites wide (so that no holes exist).  <br/>

This function sets up a 3D model of a cell with 3 compartments: Extracellular (named "default"), membrane (named "membrane").

In [ ]:
# Set up cell geometry
_ = sim.buildCapsidCell(length=micron(1.8), diameter=micron(0.5), membraneThickness=nm(64))

The same reactions from the previous tutorial are used, but this time we use a pySTDLM function to define them.

In [ ]:
# Add the reactants and reactions
_ = addLacTwoStateSystem(sim)

In [ ]:
# Get handles to the different regions
mem='membrane'
cyt='cytoplasm'
ext='default'

Being that cells are on average 20-50% full by volume, the diffusion rates of proteins and other particles are generally much lower than they would be in, for instance, water. Estimates for diffusion rates based on chain length or measured values from *in vitro* measurements will likely be much larger than in the context of a crowded cell.

To mimic the crowded context of a cell Lattice Microbes has a feature that allows it to generate a biologically occluded volume. The features allows the user to specify sets of particles (e.g. proteins) along with their sizes (radius) and counts that will be used to fill a specified region.

We have bundled up reasonable estimates for fast and slow growing *E. coli* cells into pyLM ([packFastGrowingEcoli(...)](http://www.scs.illinois.edu/schulten/lm/documentation/namespacepy_s_t_d_l_m_1_1_standard_cells.html#af9e71209a4d93c4f5b729f3c804312f) and [packSlowGrowingEcoli(...)](http://www.scs.illinois.edu/schulten/lm/documentation/namespacepy_s_t_d_l_m_1_1_standard_cells.html#aeac4afd789be4cfc4ee2ca5518a208de)). **This process uses a random search and place algorithm, so the following command can take a couple minutes to complete.**

In [ ]:
# Crowd the cytoplasm
_ = packFastGrowingEcoli(sim)

In [ ]:
# Populate the model with particles
sim.addParticles(species='R2', region=cyt,  count=9)
sim.addParticles('O',                 cyt,        1)
sim.addParticles('Y',                 mem,       30)
sim.addParticles('I',                 cyt,   7224*2)
_ = sim.addParticles('Iex',           ext,   7224*2)

In addition to saving the species counts, Lattice Microbes also saves the lattice. Generally, you will want to save the lattice less frequently, as it will take significantly more disk space.  However, make sure that the lattice write interval is divisible by the normal write interval. 

In [ ]:
# Set up the times
sim.setTimestep(microsecond(50))
sim.setWriteInterval(1)
sim.setLatticeWriteInterval(1)
sim.setSimulationTime(7200)

We have provided you with prerun simulations, so don't actually save the file. These simulations generally take about 24 hours to run.

In [ ]:
# Save the simulation state to a file
# filename='T2.2-lac2state_16uM.lm'
# sim.save(filename)

In [ ]:
# Interrogate simulation
sim

In [ ]:
# Run the simulation for 10 hours
reps=8

<h2> Analyzing the RDME Simulation </h2>

Because RDME simulations run about real time, it will take too long to actually run the simulations in the workshop. We have therefore provided pre-run simulation output for 4 different concentrations. Try examing the difference between them using the following post-processing commands.

In [ ]:
# Post-processing
filename = "T2.2-lac2state_0uM.lm"
# filename = "T2.2-lac2state_8uM.lm"
# filename = "T2.2-lac2state_16uM.lm"
# filename = "T2.2-lac2state_24uM.lm"


filehandle = openLMFile(filename)
for i in range(1,reps+1):
    plotTrace(filehandle, ['mY','Y'], i)
    plt.show()
closeLMFile(filehandle)

<h2> Assignment 3 </h2>

1. Compute the average time the operator is repressed at the different concentrations. (Hint: consider the bound (R2O, IR2O, I2R2O) and unbound (O) operators).

2. What effect do you think packing has on the solution?

3. (Challenge) Can you compute the average burst frequency per cell cycle?  Can you compute the average burst size  You can read about transcriptional bursting [here](https://en.wikipedia.org/wiki/Transcriptional_bursting).
 